## **Audio File Recognizer**

In [16]:
# Install Vosk and SoundFile
!pip install vosk soundfile

# Install FFmpeg (audio conversion tool)
!sudo apt update && sudo apt install ffmpeg -y


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 84.0 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=26fb46b0f323583870ccd59a21b345c2f5ba06764aeb8da6067fa0716c9b4ed5
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,809 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes

In [18]:
from google.colab import files

# Upload .mp3 file
uploaded = files.upload()

# Get uploaded file name
audio_file = list(uploaded.keys())[0]
print("✅ Uploaded file:", audio_file)


Saving 1cf16e15fd354af7f23806cd9e30922d48213cb1a4106f5bc8ba10424852ae66e443c0b32bc59d3f-v2.mp3 to 1cf16e15fd354af7f23806cd9e30922d48213cb1a4106f5bc8ba10424852ae66e443c0b32bc59d3f-v2.mp3
✅ Uploaded file: 1cf16e15fd354af7f23806cd9e30922d48213cb1a4106f5bc8ba10424852ae66e443c0b32bc59d3f-v2.mp3


In [19]:
# Convert to WAV format required by Vosk
!ffmpeg -i "$audio_file" -ar 16000 -ac 1 -f wav audio.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [20]:
# Download Hindi speech model (small version)
!wget -O model-hi.zip https://alphacephei.com/vosk/models/vosk-model-small-hi-0.22.zip

# Unzip the model
!unzip -o model-hi.zip


--2025-07-17 13:46:37--  https://alphacephei.com/vosk/models/vosk-model-small-hi-0.22.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44458845 (42M) [application/zip]
Saving to: ‘model-hi.zip’

model-hi.zip        100%[===================>]  42.40M  21.1MB/s    in 2.0s    

2025-07-17 13:46:39 (21.1 MB/s) - ‘model-hi.zip’ saved [44458845/44458845]

Archive:  model-hi.zip
  inflating: vosk-model-small-hi-0.22/graph/phones/word_boundary.int  
  inflating: vosk-model-small-hi-0.22/graph/Gr.fst  
  inflating: vosk-model-small-hi-0.22/graph/HCLr.fst  
  inflating: vosk-model-small-hi-0.22/graph/disambig_tid.int  
  inflating: vosk-model-small-hi-0.22/am/final.mdl  
  inflating: vosk-model-small-hi-0.22/README  
  inflating: vosk-model-small-hi-0.22/conf/model.conf  
  inflating: vosk-model-small-hi-0.22/conf/mfcc.conf  


### **Hindi Transcription Model**

In [21]:
from vosk import Model, KaldiRecognizer
import wave
import json

# Load the Hindi Vosk model
model = Model("vosk-model-small-hi-0.22")

# Open converted WAV file
wf = wave.open("audio.wav", "rb")

# Initialize recognizer
rec = KaldiRecognizer(model, wf.getframerate())
rec.SetWords(True)  # Optional: get word-level results

# Accumulate transcription
text = ""
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        result = json.loads(rec.Result())
        text += result.get("text", "") + " "

# Get final bits
final_result = json.loads(rec.FinalResult())
text += final_result.get("text", "")

# Print final transcription
print("\nFinal Hindi Transcription:\n")
print(text.strip())



Final Hindi Transcription:

हलो हलो हज़ार मीटर की लिए अप्लाई बिजली व अन्य बिजली का मीटर प्लाई या अपने आगे आगे काश या नहीं अभी कहां पर आना जी ये कृष्णा कॉलोनी में आना था हैं वह भले नंबर नौ था वहीं अब आगे आगे आ वहीं और क्या हाँ में हुई हो थी ठीक लिए आपके पास अभी नहीं बढ़ी हुई है नहीं सीख बंगलूर वन पास आया तबादलों लोग दुकान से आप बड़ा विद्वान जाकर अहलावत ने शालाओ जनाक्रोश है नई फैलाव लेकिन उसका जो हाँ बबलू का वाले गली के सामने टी प्वाइंट आएगा करना के प्वाइंट्स हैं आप का रायता जरूर जाना था राय सारे मुद्दे का थोड़ा सा आगे जाकर वो फ्रंट शायद पर है ठीक है वहीं अगर खोलकर करें आपको आर्थिक


sentiments checking

In [22]:
!pip install transformers
!pip install torch




### **Simple Sentiment Checker**



In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load public multilingual sentiment model
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Hindi-compatible sentiment analysis
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=1)
        label = torch.argmax(probs).item()

    labels = ["नकारात्मक (Negative)", "तटस्थ (Neutral)", "सकारात्मक (Positive)"]
    return labels[label], probs.tolist()[0]

# Example: Use your Vosk transcribed text here
# Replace 'text' with your actual Hindi transcription
sentiment_label, confidence = analyze_sentiment(text.strip())

# Display result
print("\nSentiment:\n", sentiment_label)
print("Confidence scores:", confidence)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Sentiment:
 तटस्थ (Neutral)
Confidence scores: [0.26500794291496277, 0.4485008716583252, 0.28649115562438965]


### **Scores According to Each Sentiment**

In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load emotion model (can handle basic multilingual inputs)
emotion_model = AutoModelForSequenceClassification.from_pretrained(
    "j-hartmann/emotion-english-distilroberta-base"
)
emotion_tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")

# Define emotion labels
emotion_labels = [
    "anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"
]

# Function to detect emotion
def detect_emotion(text):
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = emotion_model(**inputs).logits
        probs = F.softmax(logits, dim=1)
        top_prob, top_idx = torch.max(probs, dim=1)
    return emotion_labels[top_idx.item()], probs[0].tolist()

# Use the Vosk transcription here (replace `text` with your actual transcription)
emotion_label, emotion_scores = detect_emotion(text.strip())

# Display result
print("Emotion Detected:", emotion_label)
print("Emotion Scores:", dict(zip(emotion_labels, [round(score, 3) for score in emotion_scores])))


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Emotion Detected: neutral
Emotion Scores: {'anger': 0.118, 'disgust': 0.04, 'fear': 0.184, 'joy': 0.028, 'neutral': 0.49, 'sadness': 0.11, 'surprise': 0.029}
